In [1]:
import pandas as pd

# Cambia la ruta al archivo .tsv que quieras revisar
ruta_tsv = r"C:\Users\Usuario\tensorflow_datasets\downloads\extracted\TAR_GZ.xtreme_translations_FLEURS102_es_419zABzC0m-uxtwcFrs0GZfN0ebtUEOVUQlH9TtrcgEwLo.tar.gz\es_419\train.tsv"

# Cargar el archivo .tsv
df = pd.read_csv(ruta_tsv, sep='\t')

# Mostrar las primeras filas del archivo para revisar su estructura
print(df.head())

# Mostrar las columnas del archivo
print(df.columns)


   1265  13416824680170590114.wav  \
0   301  14392403388902777046.wav   
1   101   6854560521352307280.wav   
2  1198  16674325969320761881.wav   
3  1372  13262819482872106384.wav   
4  1342  17630356324166062808.wav   

  En países desarrollados, es poco frecuente escuchar niveles parecidos de reclamos sobre la calidad del agua o el derrumbe de puentes.  \
0  Después del proceso, HJR-3 será examinado una ...                                                                                      
1  Al abrirse las válvulas, se produjo la liberac...                                                                                      
2  Michael Schumacher abandonó la carrera no much...                                                                                      
3  Durante el siglo XII su estatus decayó, sobre ...                                                                                      
4  Durante el día, su temperatura es más baja que...                              

In [1]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow_datasets as tfds
from IPython import display

# Set the seed value for experiment reproducibility.
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [2]:
# Cargo los subdatasets
es_dataset, es_info = tfds.load('xtreme_s/fleurs.es_419', split = 'train', with_info=True, as_supervised=True)
en_dataset, en_info = tfds.load('xtreme_s/fleurs.en_us', split = 'train', with_info=True, as_supervised=True)
fr_dataset, fr_info = tfds.load('xtreme_s/fleurs.fr_fr', split = 'train', with_info=True, as_supervised=True)
ja_dataset, ja_info = tfds.load('xtreme_s/fleurs.ja_jp', split = 'train', with_info=True, as_supervised=True)

# De los subdatasets me quedo solo con train ya que tiene muchos ejemplos (es innecesario traer los otros ejemplos tambien) y armo los ejemplos (audio, idioma)
def add_language_label(dataset, label):
    dataset = dataset.map(lambda audio, _: (audio, label))
    return dataset

es_dataset = add_language_label(es_dataset, 0)
en_dataset = add_language_label(en_dataset, 1)
fr_dataset = add_language_label(fr_dataset, 2)
ja_dataset = add_language_label(ja_dataset, 3)

# Ver cual de los subdataset tiene menor data y de cada subdataset tomar esa cantidad de ejemplos (para balancear)
num_audios_es = len(es_dataset)
num_audios_en = len(en_dataset)
num_audios_fr = len(fr_dataset)
num_audios_ja = len(ja_dataset)

print(f"Número de audios en el dataset español: {num_audios_es}")
print(f"Número de audios en el dataset inglés: {num_audios_en}")
print(f"Número de audios en el dataset francés: {num_audios_fr}")
print(f"Número de audios en el dataset japonés: {num_audios_ja}")

# Tomamos la misma cantidad de audios para todos los datasets
cantidad = min(num_audios_es, num_audios_en, num_audios_fr, num_audios_ja)

es_dataset = es_dataset.take(cantidad)
en_dataset = en_dataset.take(cantidad)
fr_dataset = fr_dataset.take(cantidad)
ja_dataset = ja_dataset.take(cantidad)

# Combinamos los subdatasets
combined_dataset = es_dataset.concatenate(en_dataset).concatenate(fr_dataset).concatenate(ja_dataset)

# Hacemos shuffle del combined_dataset
combined_dataset = combined_dataset.shuffle(buffer_size=10000)

Número de audios en el dataset español: 2796
Número de audios en el dataset inglés: 2602
Número de audios en el dataset francés: 3193
Número de audios en el dataset japonés: 2292


In [4]:
# Ruta donde se guardará el archivo
output_file = r"C:\Users\Usuario\OneDrive\Documentos\AA2\combined_dataset\combined_dataset.tfrecord"

# Función para serializar los datos (audio e idioma)
def serialize_example(audio, label):
    # Crear un ejemplo de TensorFlow
    feature = {
        "audio": tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.serialize_tensor(audio).numpy()])),
        "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

# Guardar el dataset combinado en un archivo TFRecord
with tf.io.TFRecordWriter(output_file) as writer:
    for audio, label in combined_dataset:
        example = serialize_example(audio, label)
        writer.write(example)

print(f"Dataset combinado guardado en {output_file}")


: 